In [ ]:
# 必要なライブラリのインポート．
from abc import ABC, abstractmethod
import os
import glob
from collections import deque
from time import time
from datetime import timedelta
import pickle
from base64 import b64encode
import math
import numpy as np
import torch
from torch import nn
from torch.distributions import Normal
import torch.nn.functional as F
import gym
import matplotlib.pyplot as plt
from IPython.display import HTML

# Gymの警告を一部無視する．
gym.logger.set_level(40)
# matplotlibをColab上で描画するためのコマンド．
%matplotlib inline

In [ ]:
# pipを用いてPyBulletをインストール．
!pip install pybullet
import pybullet_envs
import pybullet as p
p.connect(p.DIRECT)

0

In [ ]:
!nvidia-smi

Tue Nov 16 13:40:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.cuda.is_available()

True

In [ ]:
torch.__version__

'1.10.0+cu111'

In [ ]:
def wrap_monitor(env):
    """ Gymの環境をmp4に保存するために，環境をラップする関数． """
    return gym.wrappers.Monitor(env, '/tmp/monitor', video_callable=lambda x: True, force=True)

def play_mp4():
    """ 保存したmp4をHTMLに埋め込み再生する関数． """
    path = glob.glob(os.path.join('/tmp/monitor', '*.mp4'))[0]
    mp4 = open(path, 'rb').read()
    url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""<video width=400 controls><source src="%s" type="video/mp4"></video>""" % url)

In [ ]:
env = wrap_monitor(gym.make('HalfCheetahBulletEnv-v0'))

print('observation space: ', env.observation_space)
print('action space: ', env.action_space)

env.reset()
done = False

# 終了シグナル(done=True)が返ってくるまで，ランダムに環境を動かす．
while (not done):
  action = env.action_space.sample()
  _, _, done, _ = env.step(np.zeros_like(action))

del env
play_mp4()

observation space:  Box(-inf, inf, (26,), float32)
action space:  Box(-1.0, 1.0, (6,), float32)


In [ ]:
class Trainer:

    def __init__(self, env, env_test, algo, seed=0, num_steps=10**6, eval_interval=10**4, num_eval_episodes=3):

        self.env = env
        self.env_test = env_test
        self.algo = algo

        # 環境の乱数シードを設定する．
        self.env.seed(seed)
        self.env_test.seed(2**31-seed)

        # 平均収益を保存するための辞書．
        self.returns = {'step': [], 'return': []}

        # データ収集を行うステップ数．
        self.num_steps = num_steps
        # 評価の間のステップ数(インターバル)．
        self.eval_interval = eval_interval
        # 評価を行うエピソード数．
        self.num_eval_episodes = num_eval_episodes

    def train(self):
        """ num_stepsステップの間，データ収集・学習・評価を繰り返す． """

        # 学習開始の時間
        self.start_time = time()
        # エピソードのステップ数．
        t = 0

        # 環境を初期化する．
        state = self.env.reset()

        for steps in range(1, self.num_steps + 1):
            # 環境(self.env)，現在の状態(state)，現在のエピソードのステップ数(t)，今までのトータルのステップ数(steps)を
            # アルゴリズムに渡し，状態・エピソードのステップ数を更新する．
            state, t = self.algo.step(self.env, state, t, steps)

            # アルゴリズムが準備できていれば，1回学習を行う．
            if self.algo.is_update(steps):
                self.algo.update()

            # 一定のインターバルで評価する．
            if steps % self.eval_interval == 0:
                self.evaluate(steps)

    def evaluate(self, steps):
        """ 複数エピソード環境を動かし，平均収益を記録する． """

        returns = []
        for _ in range(self.num_eval_episodes):
            state = self.env_test.reset()
            done = False
            episode_return = 0.0

            while (not done):
                action = self.algo.exploit(state)
                state, reward, done, _ = self.env_test.step(action)
                episode_return += reward

            returns.append(episode_return)

        mean_return = np.mean(returns)
        self.returns['step'].append(steps)
        self.returns['return'].append(mean_return)

        print(f'Num steps: {steps:<6}   '
              f'Return: {mean_return:<5.1f}   '
              f'Time: {self.time}')

    def visualize(self):
        """ 1エピソード環境を動かし，mp4を再生する． """
        env = wrap_monitor(gym.make(self.env.unwrapped.spec.id))
        state = env.reset()
        done = False

        while (not done):
            action = self.algo.exploit(state)
            state, _, done, _ = env.step(action)

        del env
        return play_mp4()

    def plot(self):
        """ 平均収益のグラフを描画する． """
        fig = plt.figure(figsize=(8, 6))
        plt.plot(self.returns['step'], self.returns['return'])
        plt.xlabel('Steps', fontsize=24)
        plt.ylabel('Return', fontsize=24)
        plt.tick_params(labelsize=18)
        plt.title(f'{self.env.unwrapped.spec.id}', fontsize=24)
        plt.tight_layout()

    @property
    def time(self):
        """ 学習開始からの経過時間． """
        return str(timedelta(seconds=int(time() - self.start_time)))

In [ ]:
class Algorithm(ABC):

    def explore(self, state):
        """ 確率論的な行動と，その行動の確率密度の対数 \log(\pi(a|s)) を返す． """
        state = torch.tensor(state, dtype=torch.float, device=self.device).unsqueeze_(0)
        with torch.no_grad():
            action, log_pi = self.actor.sample(state)
        return action.cpu().numpy()[0], log_pi.item()

    def exploit(self, state):
        """ 決定論的な行動を返す． """
        state = torch.tensor(state, dtype=torch.float, device=self.device).unsqueeze_(0)
        with torch.no_grad():
            action = self.actor(state)
        return action.cpu().numpy()[0]

    @abstractmethod
    def is_update(self, steps):
        """ 現在のトータルのステップ数(steps)を受け取り，アルゴリズムを学習するか否かを返す． """
        pass

    @abstractmethod
    def step(self, env, state, t, steps):
        """ 環境(env)，現在の状態(state)，現在のエピソードのステップ数(t)，今までのトータルのステップ数(steps)を
            受け取り，リプレイバッファへの保存などの処理を行い，状態・エピソードのステップ数を更新する．
        """
        pass

    @abstractmethod
    def update(self):
        """ 1回分の学習を行う． """
        pass

## 3.Proximal Policy Optimization(PPO)の実装

#### 3.1 [演習] 方策計算のための関数の実装

今回の演習では **共分散行列が対角なガウス分布に $\tanh$ を適用した確率分布** をPPO・SACの方策として用います．以下では，共分散行列が対角なガウス分布を単にガウス分布と呼び，共分散行列の対角成分の平方根を単に標準偏差と呼ぶことにします．また，括弧で囲んだ表記 `(...)` は，TensorのSizeを表すことにします．

方策は，学習時に探索を行うための**確率論的な行動選択**と，評価時に最適な行動を行うための**決定論的な行動選択**の2種類の行動選択を行います．確率論的な行動選択ではガウス分布からのサンプルに $\tanh$ を適用したものを，決定論的な行動選択ではガウス分布の最頻値(平均)に $\tanh$ を適用したものを行動とします．

まず，確率論的な行動を計算した際の行動の確率密度の対数 $\log \pi(a|s)$ を求める関数 `calculate_log_pi(log_stds, noises, actions)` を実装しましょう．ただし，引数の `log_stds` (標準偏差の対数)，`noises` (Reparametrization Trickにおける標準ガウス分布からのノイズ)，`actions` (行動)はすべて `(batch_size, |A|)` とし，行動の確率密度の対数は `(batch_size, 1)` で返します．



(ヒント) Reparameterization Trickでは，標準ガウス分布からのノイズ $\epsilon \sim \mathcal N(0, I)$ を用いて，平均 $\mu$，標準偏差 $\sigma$ からのサンプルを以下のように計算します．

$$
u = \mu + \epsilon * \sigma
$$

確率密度関数は平行移動に関して不変なので，ガウス分布からのサンプル $u$ の確率密度 $p(u|s)$ は $\mathcal N(0, \sigma I)$ における $\epsilon * \sigma$ の確率密度として計算することができます．その後，$\tanh$ による確率密度の変化を以下のように修正してあげましょう．

$$
\log\pi(a|s) = \log p(u|s) - \sum_{i=1}^{|\mathcal A|} \log (1 - \tanh^{2}(u_i))
$$

In [ ]:
def calculate_log_pi(log_stds, noises, actions):
    """ 確率論的な行動の確率密度を返す． """
    # ガウス分布 `N(0, stds * I)` における `noises * stds` の確率密度の対数(= \log \pi(u|a))を計算する．
    # (torch.distributions.Normalを使うと無駄な計算が生じるので，下記では直接計算しています．)
    gaussian_log_probs = \
        (-0.5 * noises.pow(2) - log_stds).sum(dim=-1, keepdim=True) - 0.5 * math.log(2 * math.pi) * log_stds.size(-1)

    # tanh による確率密度の変化を修正する．
    log_pis = gaussian_log_probs - torch.log(1 - actions.pow(2) + 1e-6).sum(dim=-1, keepdim=True)

    return log_pis

次に，Reparameterization Trickを用いて，確率的な行動 $a = \tanh(\mu + \epsilon * \sigma)$ とその行動の確率密度の対数 $\log \pi(a|s)$ を計算する関数 `reparameterize(means, log_stds)` を実装しましょう．ただし，引数の `means` (平均)と `log_stds` (標準偏差の対数)は `(batch_size, |A|)` とし，行動は `(batch_size, |A|)`，確率密度の対数は `(batch_size, 1)` で返します．



In [ ]:
def reparameterize(means, log_stds):
    """ Reparameterization Trickを用いて，確率論的な行動とその確率密度を返す． """
    # 標準偏差．
    stds = log_stds.exp()
    # 標準ガウス分布から，ノイズをサンプリングする．
    noises = torch.randn_like(means)
    # Reparameterization Trickを用いて，N(means, stds)からのサンプルを計算する．
    us = means + noises * stds
    # tanh　を適用し，確率論的な行動を計算する．
    actions = torch.tanh(us)

    # 確率論的な行動の確率密度の対数を計算する．
    log_pis = calculate_log_pi(log_stds, noises, actions)

    return actions, log_pis

また，のちのち簡単に方策を実装できるように，ある平均・標準偏差の対数でパラメータ化したガウス分布 + $\tanh$ の方策における，ある行動の確率密度の対数を計算する関数 `evaluate_lop_pi(means, log_stds, actions)` をあらかじめ定義しておきます．

In [ ]:
def atanh(x):
    """ tanh の逆関数． """
    return 0.5 * (torch.log(1 + x + 1e-6) - torch.log(1 - x + 1e-6))


def evaluate_lop_pi(means, log_stds, actions):
    """ 平均(mean)，標準偏差の対数(log_stds)でパラメータ化した方策における，行動(actions)の確率密度の対数を計算する． """
    noises = (atanh(actions) - means) / (log_stds.exp() + 1e-8)
    return calculate_log_pi(log_stds, noises, actions)

### 3.2 ネットワークの実装

PPOの方策では，ユニット数64の隠れ層を2層もち，中間層の活性化関数に $\tanh$ を用いたネットワークを構築します．このネットワークは，入力として状態を受け取り，**ガウス分布の平均**を出力します．また**ガウス分布の標準偏差の対数**を，学習するパラメータとして保持します．

以下の4つのメソッドを持つ，PPOの方策を関数近似するネットワークのクラス `PPOActor` を実装します．

- `__init__(self, state_shape, action_shape)`

> 入力として状態を受け取り，**ガウス分布の平均**を出力するネットワークを構築します．また，**ガウス分布の標準偏差の対数**を表すパラメータを作成します．

- `forward(self, states)`

> `(batch_size, |S|)` の `states` を受け取り，決定論な行動 $a$ を `(batch_size, 1)`で返します．

- `sample(self, states)`

> `(batch_size, |S|)` の `states` を受け取り，確率論的な行動 $a$ とその行動の確率密度の対数 $\log(\pi(a|s))$ をそれぞれ `(batch_size, 1)` で返します．

- `evaluate_log_pi(self, states, actions)`

> `(batch_size, |S|)` の `states` と，`(batch_size, |A|)` の `actions` を受け取り，現在の方策における行動 `actions` の確率密度の対数を `(batch_size, 1)` で返します．


In [ ]:
class PPOActor(nn.Module):

    def __init__(self, state_shape, action_shape):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(state_shape[0], 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, action_shape[0]),
        )
        self.log_stds = nn.Parameter(torch.zeros(1, action_shape[0]))

    def forward(self, states):
        return torch.tanh(self.net(states))

    def sample(self, states):
        return reparameterize(self.net(states), self.log_stds)

    def evaluate_log_pi(self, states, actions):
        return evaluate_lop_pi(self.net(states), self.log_stds, actions)

続いて，PPOの状態価値を関数近似するネットワークのクラス `PPOCritic` を実装します．このネットワークも，ユニット数64の隠れ層を2層もち，中間層の活性化関数に $\tanh$ を用います．入力として状態を受け取り，状態価値を出力します．

- `__init__(self, state_shape)`

> 入力として状態受け取り，状態価値を出力するネットワークを構築します．

- `forward(self, states)`

> `(batch_size, |S|)` の `states` を受け取り，状態価値を `(batch_size, 1)` で返します．




In [ ]:
class PPOCritic(nn.Module):

    def __init__(self, state_shape):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(state_shape[0], 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 1),
        )

    def forward(self, states):
        return self.net(states)

### 3.3 Generalized Advantage Estimation(GAE)の実装

PPOでは，アドバンテージの推定に**Generalized Advantage Estimation**(GAE)[[4]](#scrollTo=HOq7n-OJboPr)を用います．GAEではnステップのアドバンテージ $\hat A_t^{(n)} = r_t + \gamma r_{t+1} + \cdots + \gamma^n V(s_{t+n}) - V(s_t)$ を用いて，以下のようにアドバンテージを推定します．

$$
\hat A_t^{GAE}(\lambda) = (1-\lambda)(\hat A_t^{(1)} + \lambda \hat A_t^{(2)} + \lambda^2 \hat A_t^{(3)} + \cdots)
$$

このとき，TD誤差 $\delta_t= r_t+ \gamma V(s_{t+1}) - V(s_t)$ を用いると，以下のように式変形できます．(発展課題で導出に挑戦してみましょう！)

$$
\hat A_t^{GAE}(\lambda) = \sum_{i=0}^{\infty} (\gamma \lambda)^i \delta_{t+i}
$$

従って，GAEは再帰的に計算することが可能です．ただし，実際にはロールアウト長を $T$ としたときに，$t = T+1$ 以降のGAEをすべて $0$ と近似して計算を行います．

$$
\hat A_t^{GAE}(\lambda) = \delta_t + (\gamma \lambda) \hat A_{t+1}^{GAE}(\lambda)
$$

\\
またPPOでは，状態価値のターゲットを $\lambda$-収益 $R_t(\lambda)$ を用いて推定します．

$$
R_t(\lambda) = \hat A_t^{GAE}(\lambda) + V(s_t)
$$


In [ ]:
def calculate_advantage(values, rewards, dones, next_values, gamma=0.995, lambd=0.997):
    """ GAEを用いて，状態価値のターゲットとGAEを計算する． """

    # TD誤差を計算する．
    deltas = rewards + gamma * next_values * (1 - dones) - values

    # GAEを初期化する．
    advantages = torch.empty_like(rewards)

    # 終端ステップを計算する．
    advantages[-1] = deltas[-1]

    # 終端ステップの1つ前から，順番にGAEを計算していく．
    for t in reversed(range(rewards.size(0) - 1)):
        advantages[t] = deltas[t] + gamma * lambd * (1 - dones[t]) * advantages[t + 1]

    # 状態価値のターゲットをλ-収益として計算する．
    targets = advantages + values

    # GAEを標準化する．
    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

    return targets, advantages

### 3.4 [演習] 学習アルゴリズム(PPO)の実装

まず，収集したデータを保存するためのバッファを用意します．ここでは，時刻tの状態・行動・即時報酬・終了シグナル・確率密度の対数，そして時刻t+1の状態をロールアウト1回分保存することとします．

In [ ]:
class RolloutBuffer:

    def __init__(self, buffer_size, state_shape, action_shape, device=torch.device('cuda')):

        # GPU上に保存するデータ．
        self.states = torch.empty((buffer_size, *state_shape), dtype=torch.float, device=device)
        self.actions = torch.empty((buffer_size, *action_shape), dtype=torch.float, device=device)
        self.rewards = torch.empty((buffer_size, 1), dtype=torch.float, device=device)
        self.dones = torch.empty((buffer_size, 1), dtype=torch.float, device=device)
        self.log_pis = torch.empty((buffer_size, 1), dtype=torch.float, device=device)
        self.next_states = torch.empty((buffer_size, *state_shape), dtype=torch.float, device=device)

        # 次にデータを挿入するインデックス．
        self._p = 0
        # バッファのサイズ．
        self.buffer_size = buffer_size

    def append(self, state, action, reward, done, log_pi, next_state):
        self.states[self._p].copy_(torch.from_numpy(state))
        self.actions[self._p].copy_(torch.from_numpy(action))
        self.rewards[self._p] = float(reward)
        self.dones[self._p] = float(done)
        self.log_pis[self._p] = float(log_pi)
        self.next_states[self._p].copy_(torch.from_numpy(next_state))
        self._p = (self._p + 1) % self.buffer_size
    
    def get(self):
        assert self._p == 0, 'Buffer needs to be full before training.'
        return self.states, self.actions, self.rewards, self.dones, self.log_pis, self.next_states

では，いよいよPPOの学習アルゴリズムを実装していきましょう！

今回の演習では，`update_critic` と `update_actor` の2つのメソッドを実装します．以下では，方策のネットワークのパラメータを $\phi$，状態価値のネットワークのパラメータを $\theta$ とします．また，ロールアウト長を $T$ とします．

- `update_critic(self, states, targets)`

> `(batch_size, |S|)` の `states` (状態)と `(batch_size, 1)` の `targets` (状態価値のターゲット)を受け取り，Criticのネットワークを更新します．

> 状態価値のネットワークの損失関数は **平均二乗誤差** を用います．

$$
\mathcal L^{PPO}_V(\theta) = E_{t \in [1, T]}[ (V_\theta(s_t) - R_t(\lambda))^2 ]
$$

- `update_actor(self, states, actions, log_pis_old, advantages)`

> `(batch_size, |S|)` の `states` (状態)と `(batch_size, |A|)` の `actions` (行動)，そして `(batch_size, 1)` の `log_pis_old` (データ収集時の方策における行動の確率密度)と `advantages` (GAE)を受け取り，Actorのネットワークを更新します．

> 方策のネットワークの損失関数は，以下の式を用います．ただし，過去の方策のネットワークのパラメータを $\phi_{old}$ とします．また，損失関数の $\epsilon$ は `self.clip_eps` に保持されています．

$$
\mathcal L^{PPO}_\pi(\phi) = E_{t \in [1, T]}[ \min(\frac{\pi_\phi(a_t|s_t)}{\pi_{\phi_{old}}(a_t|s_t)}\hat A_t^{GAE}(\lambda), \; clip(\frac{\pi_\phi(a_t|s_t)}{\pi_{\phi_{old}}(a_t|s_t)}, 1-\epsilon, 1+\epsilon) \hat A_t^{GAE}(\lambda) ]
$$



In [ ]:
class PPO(Algorithm):

    def __init__(self, state_shape, action_shape, device=torch.device('cuda'), seed=0,
                 batch_size=64, gamma=0.995, lr_actor=3e-4, lr_critic=3e-4,
                 rollout_length=2048, num_updates=10, clip_eps=0.2, lambd=0.97,
                 coef_ent=0.0, max_grad_norm=0.5):
        super().__init__()

        # シードを設定する．
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)

        # データ保存用のバッファ．
        self.buffer = RolloutBuffer(
            buffer_size=rollout_length,
            state_shape=state_shape,
            action_shape=action_shape,
            device=device
        )

        # Actor-Criticのネットワークを構築する．
        self.actor = PPOActor(
            state_shape=state_shape,
            action_shape=action_shape,
        ).to(device)
        self.critic = PPOCritic(
            state_shape=state_shape,
        ).to(device)

        # オプティマイザ．
        self.optim_actor = torch.optim.Adam(self.actor.parameters(), lr=lr_actor)
        self.optim_critic = torch.optim.Adam(self.critic.parameters(), lr=lr_critic)

        # その他パラメータ．
        self.learning_steps = 0
        self.device = device
        self.batch_size = batch_size
        self.gamma = gamma
        self.rollout_length = rollout_length
        self.num_updates = num_updates
        self.clip_eps = clip_eps
        self.lambd = lambd
        self.coef_ent = coef_ent
        self.max_grad_norm = max_grad_norm

    def is_update(self, steps):
        # ロールアウト1回分のデータが溜まったら学習する．
        return steps % self.rollout_length == 0

    def step(self, env, state, t, steps):
        t += 1

        action, log_pi = self.explore(state)
        next_state, reward, done, _ = env.step(action)

        # ゲームオーバーではなく，最大ステップ数に到達したことでエピソードが終了した場合は，
        # 本来であればその先も試行が継続するはず．よって，終了シグナルをFalseにする．
        # NOTE: ゲームオーバーによってエピソード終了した場合には， done_masked=True が適切．
        # しかし，以下の実装では，"たまたま"最大ステップ数でゲームオーバーとなった場合には，
        # done_masked=False になってしまう．
        # その場合は稀で，多くの実装ではその誤差を無視しているので，今回も無視する．
        if t == env._max_episode_steps:
            done_masked = False
        else:
            done_masked = done

        # バッファにデータを追加する．
        self.buffer.append(state, action, reward, done_masked, log_pi, next_state)

        # エピソードが終了した場合には，環境をリセットする．
        if done:
            t = 0
            next_state = env.reset()

        return next_state, t

    def update(self):
        self.learning_steps += 1

        states, actions, rewards, dones, log_pis, next_states = self.buffer.get()

        with torch.no_grad():
            values = self.critic(states)
            next_values = self.critic(next_states)
        targets, advantages = calculate_advantage(values, rewards, dones, next_values, self.gamma, self.lambd)

        # バッファ内のデータを num_updates回ずつ使って，ネットワークを更新する．
        for _ in range(self.num_updates):
            # インデックスをシャッフルする．
            indices = np.arange(self.rollout_length)
            np.random.shuffle(indices)

            # ミニバッチに分けて学習する．
            for start in range(0, self.rollout_length, self.batch_size):
                idxes = indices[start:start+self.batch_size]
                self.update_critic(states[idxes], targets[idxes])
                self.update_actor(states[idxes], actions[idxes], log_pis[idxes], advantages[idxes])

    def update_critic(self, states, targets):
        loss_critic = (self.critic(states) - targets).pow_(2).mean()

        self.optim_critic.zero_grad()
        loss_critic.backward(retain_graph=False)
        # 学習を安定させるヒューリスティックとして，勾配のノルムをクリッピングする．
        nn.utils.clip_grad_norm_(self.critic.parameters(), self.max_grad_norm)
        self.optim_critic.step()

    def update_actor(self, states, actions, log_pis_old, advantages):
        log_pis = self.actor.evaluate_log_pi(states, actions)
        mean_entropy = -log_pis.mean()

        ratios = (log_pis - log_pis_old).exp_()
        loss_actor1 = -ratios * advantages
        loss_actor2 = -torch.clamp(
            ratios,
            1.0 - self.clip_eps,
            1.0 + self.clip_eps
        ) * advantages
        loss_actor = torch.max(loss_actor1, loss_actor2).mean() - self.coef_ent * mean_entropy

        self.optim_actor.zero_grad()
        loss_actor.backward(retain_graph=False)
        # 学習を安定させるヒューリスティックとして，勾配のノルムをクリッピングする．
        nn.utils.clip_grad_norm_(self.actor.parameters(), self.max_grad_norm)
        self.optim_actor.step()

### 3.5 実験

それでは，実装したPPOを学習させてみましょう！

#### InvertedPendulumBulletEnv-v0



まず，`InvertedPendulumBulletEnv-v0` でPPOを $3 \times 10^4$ ステップ学習させてみましょう！学習には2~3分ほどかかります．うまく学習できると，平均収益が1000に達します．

#### HalfCheetahBulletEnv-v0

次に，`HalfCheetahBulletEnv-v0` でPPOを $3 * 10^5$ ステップ学習させてみましょう！うまく学習できると，1500~1700程度の平均収益に達します．

In [ ]:
ENV_ID = 'HalfCheetahBulletEnv-v0'
SEED = 0
NUM_STEPS = 3 * 10 ** 5
EVAL_INTERVAL = 10 ** 4

env = gym.make(ENV_ID)
env_test = gym.make(ENV_ID)

algo = PPO(
    state_shape=env.observation_space.shape,
    action_shape=env.action_space.shape,
    seed=SEED
)

trainer = Trainer(
    env=env,
    env_test=env_test,
    algo=algo,
    seed=SEED,
    num_steps=NUM_STEPS,
    eval_interval=EVAL_INTERVAL,
)

In [ ]:
trainer.train()

Num steps: 10000    Return: -1303.4   Time: 0:00:42
Num steps: 20000    Return: -1322.8   Time: 0:01:28
Num steps: 30000    Return: -1284.8   Time: 0:02:13
Num steps: 40000    Return: 147.0   Time: 0:02:59
Num steps: 50000    Return: -957.1   Time: 0:03:45
Num steps: 60000    Return: -1235.2   Time: 0:04:31
Num steps: 70000    Return: 575.6   Time: 0:05:16
Num steps: 80000    Return: -595.8   Time: 0:06:01


KeyboardInterrupt: ignored

In [ ]:
trainer.plot()

In [ ]:
trainer.visualize()

In [ ]:
del env
del env_test
del algo
del trainer



参考文献

[[1]](https://arxiv.org/abs/1707.06347) Schulman, John, et al. "Proximal policy optimization algorithms." arXiv preprint arXiv:1707.06347 (2017).

[[2]](https://arxiv.org/abs/1801.01290) Haarnoja, Tuomas, et al. "Soft actor-critic: Off-policy maximum entropy deep reinforcement learning with a stochastic actor." arXiv preprint arXiv:1801.01290 (2018).

[[3]](https://arxiv.org/abs/1812.05905) Haarnoja, Tuomas, et al. "Soft actor-critic algorithms and applications." arXiv preprint arXiv:1812.05905 (2018).

[[4]](https://arxiv.org/abs/1506.02438) Schulman, John, et al. "High-dimensional continuous control using generalized advantage estimation." arXiv preprint arXiv:1506.02438 (2015).

[[5]](https://arxiv.org/abs/2006.05990) Andrychowicz, Marcin, et al. "What Matters In On-Policy Reinforcement Learning? A Large-Scale Empirical Study." arXiv preprint arXiv:2006.05990 (2020).